### Importing Libraries and Constants Definition

In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

DATA_PATH = '../data/public/'
PROPOSALS_FILE = DATA_PATH + 'brasilparticipativo.presidencia.gov.br-open-data-proposals.csv'
VOCAB_FILE = DATA_PATH + 'vocabulario-controlado-basico-vcb-lista-alfabetica.txt'
TOP_N_TOPICS = 5

### Functions Definition

In [15]:
def process_vocab_file(vocab_file):
    ROWS_TO_SKIP = 8
    vocab_dict = {}
    with open(vocab_file, 'r', encoding='utf-8') as file:
        current_term = ''
        for line in file.read().splitlines()[ROWS_TO_SKIP:]:
            if not line.startswith('\t') and line != '':
                current_term = line
                vocab_dict[current_term] = {
                    'USE': [], # Use
                    'DF': [],  # Definição
                    'UP': [],  # Termos não preferenciais
                    'TG': [],  # Termo geral
                    'TR': [],  # Termos relacionados
                    'TE': [],  # Termos específicos
                    'EQ': [],  # Equivalente
                    'Nota de escopo': [],
                }
            elif line != '':
                property = line.strip().split(':')
                key = property[0].strip()
                value = property[1].strip()
                vocab_dict[current_term][key].append(value)
    return vocab_dict

def process_vocab(vocab_file, exclusion_terms):
    vocab_json = process_vocab_file(vocab_file)
    vocab_list_processed = [key for key in vocab_json if key.lower() not in exclusion_terms]
    return vocab_list_processed

def load_and_preprocess_proposals(file_path):
    df = pd.read_csv(file_path, delimiter=';')
    # df = df[~df['state'].isin(['rejected', 'withdrawn'])]
    df['body/pt-BR'] = df['body/pt-BR'].str.replace(r'<[^>]*>', '', regex=True)
    relevant_cols = [
        'id',
        'category/id',
        'category/name/pt-BR',
        'title/pt-BR',
        'body/pt-BR',
        'supports',
        'followers',
        'comments',
        'published_at',
        'state', # REMOVE
        'url',
        'participatory_space/url',
    ]
    return df[relevant_cols]

def get_unique_themes(df):
    themes = df['category/name/pt-BR'].str.lower().unique().tolist()
    themes = [str(theme) for theme in themes if theme is not np.nan]
    themes = set([theme.split('- direito à')[-1].strip() for theme in themes])
    themes = set([theme.split('- direito ao')[-1].strip() for theme in themes])
    return themes


def classify_topics(sentences, topics, top_n=TOP_N_TOPICS):
    all_text = sentences + topics
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_text)

    classified_data = []
    for i, sentence in enumerate(sentences):
        cosine_similarities = cosine_similarity(tfidf_matrix[i:i+1], tfidf_matrix[len(sentences):]).flatten()
        top_n_indices = np.argsort(cosine_similarities)[-top_n:][::-1]
        top_n_topic_similarities = [(topics[idx], cosine_similarities[idx]) for idx in top_n_indices]
        
        # Append a tuple containing the sentence and its classified topics
        classified_data.append((sentence, top_n_topic_similarities))

    return classified_data

### Loading and preprocessing proposals

In [16]:
df_propostas = load_and_preprocess_proposals(PROPOSALS_FILE)
df_propostas.head(5)

,id,category/id,category/name/pt-BR,title/pt-BR,body/pt-BR,supports,followers,comments,published_at,state,url,participatory_space/url
0,1,30.0,Turismo,Turismo: esse é o Destino,Objetivo: Posicionar o turismo como vetor de d...,1,1,0,2023-05-10 10:03:41 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...
1,8,31.0,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia,Objetivo: Fortalecer a agricultura familiar em...,1,0,0,2023-05-10 16:22:51 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...
2,9,1.0,Agricultura e Pecuária,Agropecuária Sustentável,Objetivo: Contribuir para o desenvolvimento do...,2,0,0,2023-05-10 16:35:47 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...
3,10,27.0,Saúde,Atenção Primária à Saúde,"Fortalecer a Atenção Primária à Saúde, amplian...",20427,515,0,2023-05-10 16:42:43 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...
4,11,27.0,Saúde,Atenção Especializada à Saúde,Ampliar o acesso às ações e serviços da Atençã...,18786,383,0,2023-05-10 16:41:01 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...


### Extracting sentences from proposals and titles

In [17]:
df_propostas['sentence'] = df_propostas['title/pt-BR'].str.lower() + ' ' + df_propostas['body/pt-BR'].str.lower()
sentences = df_propostas['sentence'].tolist()
sentences[:5]

['turismo: esse é o destino objetivo: posicionar o turismo como vetor de desenvolvimento sustentável e aumentar a competitividade dos destinos e produtos turísticos brasileiros, democratizando o acesso à atividade turística aos cidadãos brasileiros.órgão responsável: ministério do turismopágina oficial: https://www.gov.br/turismo/pt-br/',
 'agricultura familiar e agroecologia objetivo: fortalecer a agricultura familiar em sua diversidade e a agroecologia, promovendo a inclusão socioeconômica, com fomento à produção sustentável e à geração de renda, contribuindo para a promoção da segurança alimentar e nutricional e da vida digna, com redução das desigualdades e mitigação das mudanças climáticas.órgão responsável: ministério do desenvolvimento agráriopágina oficial: https://www.gov.br/agricultura/pt-br/assuntos/mda/agricultura-familiar-1',
 'agropecuária sustentável objetivo: contribuir para o desenvolvimento do setor agropecuário, com sustentabilidade ambiental, econômica e social, por

### Getting unique themes

In [18]:
themes = get_unique_themes(df_propostas)
list(themes)[:5]

['meio ambiente e mudança do clima',
 'sustentabilidade e ao meio ambiente',
 'segurança institucional',
 'xii - sistema nacional de juventude - sinajuve',
 'previdência social']

### Processing vocabulary

In [19]:
vocab_list = process_vocab(VOCAB_FILE, themes)
vocab_list[:10]

['2019-nCoV disease',
 '2FA',
 '2Factor Authentication',
 'AAE',
 'Ab-rogação',
 'Abacate',
 'Abacaxi',
 'Abalo sísmico',
 'Abalroamento aéreo',
 'Abalroamento de navios']

### Classifying topics

In [20]:
classified_topics = classify_topics(sentences, vocab_list)
classified_topics[:3]

[('turismo: esse é o destino objetivo: posicionar o turismo como vetor de desenvolvimento sustentável e aumentar a competitividade dos destinos e produtos turísticos brasileiros, democratizando o acesso à atividade turística aos cidadãos brasileiros.órgão responsável: ministério do turismopágina oficial: https://www.gov.br/turismo/pt-br/',
  [('Turismo sustentável', 0.38985294921413616),
   ('Turismo de saúde', 0.3440995848691307),
   ('Turismo social', 0.33531350226999784),
   ('Infraestrutura de turismo', 0.301708351417167),
   ('Turismo rural', 0.30070104158261457)]),
 ('agricultura familiar e agroecologia objetivo: fortalecer a agricultura familiar em sua diversidade e a agroecologia, promovendo a inclusão socioeconômica, com fomento à produção sustentável e à geração de renda, contribuindo para a promoção da segurança alimentar e nutricional e da vida digna, com redução das desigualdades e mitigação das mudanças climáticas.órgão responsável: ministério do desenvolvimento agráriopá

### Processing classified topics

In [21]:
classified_topics_processed = [[sentence] + [f"{x[0]} ({round(x[1]*100)}%)" for x in topics] for sentence, topics in classified_topics]
classified_topics_processed[:3]

[['turismo: esse é o destino objetivo: posicionar o turismo como vetor de desenvolvimento sustentável e aumentar a competitividade dos destinos e produtos turísticos brasileiros, democratizando o acesso à atividade turística aos cidadãos brasileiros.órgão responsável: ministério do turismopágina oficial: https://www.gov.br/turismo/pt-br/',
  'Turismo sustentável (39%)',
  'Turismo de saúde (34%)',
  'Turismo social (34%)',
  'Infraestrutura de turismo (30%)',
  'Turismo rural (30%)'],
 ['agricultura familiar e agroecologia objetivo: fortalecer a agricultura familiar em sua diversidade e a agroecologia, promovendo a inclusão socioeconômica, com fomento à produção sustentável e à geração de renda, contribuindo para a promoção da segurança alimentar e nutricional e da vida digna, com redução das desigualdades e mitigação das mudanças climáticas.órgão responsável: ministério do desenvolvimento agráriopágina oficial: https://www.gov.br/agricultura/pt-br/assuntos/mda/agricultura-familiar-1',

### Creating DataFrame with classified topics

In [22]:
columns = ['sentence'] + [f"Tópico {i}" for i in range(1, TOP_N_TOPICS + 1)]
df_classified = pd.DataFrame(classified_topics_processed, columns=columns)
df_classified.head(5)

,sentence,Tópico 1,Tópico 2,Tópico 3,Tópico 4,Tópico 5
0,turismo: esse é o destino objetivo: posicionar...,Turismo sustentável (39%),Turismo de saúde (34%),Turismo social (34%),Infraestrutura de turismo (30%),Turismo rural (30%)
1,agricultura familiar e agroecologia objetivo: ...,Agricultura familiar (53%),Agricultura (43%),"Agricultura, produção (38%)",Agricultura sustentável (38%),Agricultura e Estado (32%)
2,agropecuária sustentável objetivo: contribuir ...,Produção agropecuária (33%),Agropecuária (33%),"Agropecuária, projeto (27%)",Pesquisa agropecuária (26%),"Agropecuária, pesquisa (26%)"
3,atenção primária à saúde fortalecer a atenção ...,Atenção primária à saúde (46%),Atenção à saúde (39%),Saúde da família (38%),Promoção da saúde (38%),Medicamentos para a atenção primária à saúde (...
4,atenção especializada à saúde ampliar o acesso...,Atenção à saúde (38%),Serviços de saúde (31%),Direito da Saúde (31%),Atenção básica à saúde (30%),Atenção básica (saúde) (30%)


### Merging dataframes

In [23]:
df_merged = pd.merge(df_propostas, df_classified, on='sentence', how='left')
df_merged.head(5)

,id,category/id,category/name/pt-BR,title/pt-BR,body/pt-BR,supports,followers,comments,published_at,state,url,participatory_space/url,sentence,Tópico 1,Tópico 2,Tópico 3,Tópico 4,Tópico 5
0,1,30.0,Turismo,Turismo: esse é o Destino,Objetivo: Posicionar o turismo como vetor de d...,1,1,0,2023-05-10 10:03:41 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...,turismo: esse é o destino objetivo: posicionar...,Turismo sustentável (39%),Turismo de saúde (34%),Turismo social (34%),Infraestrutura de turismo (30%),Turismo rural (30%)
1,8,31.0,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia,Objetivo: Fortalecer a agricultura familiar em...,1,0,0,2023-05-10 16:22:51 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...,agricultura familiar e agroecologia objetivo: ...,Agricultura familiar (53%),Agricultura (43%),"Agricultura, produção (38%)",Agricultura sustentável (38%),Agricultura e Estado (32%)
2,8,31.0,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia,Objetivo: Fortalecer a agricultura familiar em...,1,0,0,2023-05-10 16:22:51 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...,agricultura familiar e agroecologia objetivo: ...,Agricultura familiar (53%),Agricultura (43%),"Agricultura, produção (38%)",Agricultura sustentável (38%),Agricultura e Estado (32%)
3,9,1.0,Agricultura e Pecuária,Agropecuária Sustentável,Objetivo: Contribuir para o desenvolvimento do...,2,0,0,2023-05-10 16:35:47 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...,agropecuária sustentável objetivo: contribuir ...,Produção agropecuária (33%),Agropecuária (33%),"Agropecuária, projeto (27%)",Pesquisa agropecuária (26%),"Agropecuária, pesquisa (26%)"
4,9,1.0,Agricultura e Pecuária,Agropecuária Sustentável,Objetivo: Contribuir para o desenvolvimento do...,2,0,0,2023-05-10 16:35:47 -0300,NaN,http://brasilparticipativo.presidencia.gov.br/...,http://brasilparticipativo.presidencia.gov.br/...,agropecuária sustentável objetivo: contribuir ...,Produção agropecuária (33%),Agropecuária (33%),"Agropecuária, projeto (27%)",Pesquisa agropecuária (26%),"Agropecuária, pesquisa (26%)"


### Identifying Participative Processes

In [24]:
conjuv_url = df_merged['participatory_space/url'].tolist()[-1]
conjuv = conjuv_url.split('/')[3] + '|' + conjuv_url.split('/')[4].split('?')[0]  + '|' + conjuv_url.split('=')[-1]
program_url = df_merged['participatory_space/url'].tolist()[0]
program = program_url.split('/')[3] + '|' + program_url.split('/')[4].split('?')[0]  + '|' + program_url.split('=')[-1]
set(df_merged['participatory_space/url'].tolist()), conjuv, program

({'http://brasilparticipativo.presidencia.gov.br/assemblies/confjuv4?assembly_slug=confjuv4',
  'http://brasilparticipativo.presidencia.gov.br/processes/programas?participatory_process_slug=programas'},
 'assemblies|confjuv4|confjuv4',
 'processes|programas|programas')

In [25]:
df_merged['participatory_space'] = df_merged['participatory_space/url'].str.split('/').str[3] + '|' + \
                                  df_merged['participatory_space/url'].str.split('/').str[4].str.split('?').str[0] + '|' + \
                                  df_merged['participatory_space/url'].str.split('/').str[4].str.split('=').str[-1]
df_merged['participatory_space']

0       processes|programas|programas
1       processes|programas|programas
2       processes|programas|programas
3       processes|programas|programas
4       processes|programas|programas
                    ...              
8947     assemblies|confjuv4|confjuv4
8948     assemblies|confjuv4|confjuv4
8949     assemblies|confjuv4|confjuv4
8950     assemblies|confjuv4|confjuv4
8951     assemblies|confjuv4|confjuv4
Name: participatory_space, Length: 8952, dtype: object

### Renaming columns

In [26]:
df_merged_renamed = df_merged[['id', 'state', 'participatory_space', 'category/name/pt-BR', 'supports', 'title/pt-BR', 'body/pt-BR', 'Tópico 1', 'Tópico 2', 'Tópico 3', 'Tópico 4', 'Tópico 5']] \
    .rename(columns={
        'id': 'id_proposta',
        'state': 'status',
        'participatory_space': 'programa',
        'category/name/pt-BR': 'tema',
        'supports': 'votos',
        'title/pt-BR': 'titulo',
        'body/pt-BR': 'proposta'
    })
df_merged_renamed

,id_proposta,status,programa,tema,votos,titulo,proposta,Tópico 1,Tópico 2,Tópico 3,Tópico 4,Tópico 5
0,1,NaN,processes|programas|programas,Turismo,1,Turismo: esse é o Destino,Objetivo: Posicionar o turismo como vetor de d...,Turismo sustentável (39%),Turismo de saúde (34%),Turismo social (34%),Infraestrutura de turismo (30%),Turismo rural (30%)
1,8,NaN,processes|programas|programas,Desenvolvimento Agrário e Agricultura Familiar,1,Agricultura Familiar e Agroecologia,Objetivo: Fortalecer a agricultura familiar em...,Agricultura familiar (53%),Agricultura (43%),"Agricultura, produção (38%)",Agricultura sustentável (38%),Agricultura e Estado (32%)
2,8,NaN,processes|programas|programas,Desenvolvimento Agrário e Agricultura Familiar,1,Agricultura Familiar e Agroecologia,Objetivo: Fortalecer a agricultura familiar em...,Agricultura familiar (53%),Agricultura (43%),"Agricultura, produção (38%)",Agricultura sustentável (38%),Agricultura e Estado (32%)
3,9,NaN,processes|programas|programas,Agricultura e Pecuária,2,Agropecuária Sustentável,Objetivo: Contribuir para o desenvolvimento do...,Produção agropecuária (33%),Agropecuária (33%),"Agropecuária, projeto (27%)",Pesquisa agropecuária (26%),"Agropecuária, pesquisa (26%)"
4,9,NaN,processes|programas|programas,Agricultura e Pecuária,2,Agropecuária Sustentável,Objetivo: Contribuir para o desenvolvimento do...,Produção agropecuária (33%),Agropecuária (33%),"Agropecuária, projeto (27%)",Pesquisa agropecuária (26%),"Agropecuária, pesquisa (26%)"
...,...,...,...,...,...,...,...,...,...,...,...,...
8947,12024,NaN,assemblies|confjuv4|confjuv4,"I - Direito à Cidadania, à Participação Social...",4,Direitos Humano,Introduzir logo nos primeiros anos do estudo f...,Ser humano (33%),Desenvolvimento humano (33%),Capital humano (28%),Crescimento humano (28%),Corpo humano (27%)
8948,12025,NaN,assemblies|confjuv4|confjuv4,X - Direito ao Território e à Mobilidade,17,"MobiMove: Não perca seu ônibus, Não perca seu ...",O projeto MobiMove foi criado a partir do Star...,Ônibus (36%),Ônibus-biblioteca (26%),Ônibus a gás (25%),Corredor de ônibus (23%),Ônibus elétrico (23%)
8949,12026,NaN,assemblies|confjuv4|confjuv4,V - Direito à Saúde,1,Política nacional de saúde mental,"Criar uma política nacional de saúde mental, c...",Saúde mental (45%),Política de saúde (42%),Direito à saúde mental (40%),"Saúde, política (40%)",Promoção da saúde mental (37%)
8950,12027,NaN,assemblies|confjuv4|confjuv4,"I - Direito à Cidadania, à Participação Social...",0,Justiça e Segurança Pública,Proposta de revisão e regulação das restrições...,Compensação de horários (23%),Concessão de obra pública (19%),Direitos das crianças (19%),Política de segurança pública (19%),Não-concessão de justiça (18%)


### Saving to file

In [27]:
OUTPUT_FILE = DATA_PATH + 'results_TopicClassification_TFIDF_CosineSimilarity.csv'
df_merged = df_merged.drop_duplicates()
df_merged.to_csv(OUTPUT_FILE, index=False)